In [1]:
# импорты
import pandas as pd
import re

df = pd.read_csv('analyzer_analyzer_urls.csv', sep=',')

In [2]:
# определим названия колонок для df
df.columns = ['html', 'html_body']

In [3]:
df.describe()

,html,html_body
count,9999,9999
unique,7788,9780
top,http://0009.ru,"<html lang=""ru""><head>\n <meta http-equ..."
freq,2,57


In [4]:
# неплохо будет избавиться от дубликатов
duplicate_rows = df[df.duplicated(keep=False)]
duplicate_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 126 entries, 20 to 9614
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   html       126 non-null    object
 1   html_body  126 non-null    object
dtypes: object(2)
memory usage: 3.0+ KB


In [5]:
# тут я решил убедиться что дубликаты полные

# def find_duplicate_rows(df):
#     duplicate_rows = df[df.duplicated(keep=False)]
#     duplicate_indices = duplicate_rows.index.tolist()  # индексы неуникальных строк
    
#     return duplicate_rows, duplicate_indices


# duplicate_rows, duplicate_indices = find_duplicate_rows(df)

# dupl_pair = df.loc[df['html'] == 'http://01bit.ru']

# dupl_pair.loc[21, 'html_body']
# with open ('1.txt', 'w') as t:
#     t.write(dupl_pair.loc[21, 'html_body'])
# with open ('2.txt', 'w') as t:
#     t.write(dupl_pair.loc[440, 'html_body'])
    
# dupl_pair


In [3]:
# удалим дубликаты 
df.drop_duplicates(subset='html_body', keep='first', inplace=True, )

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9780 entries, 0 to 9998
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   html       9780 non-null   object
 1   html_body  9780 non-null   object
dtypes: object(2)
memory usage: 229.2+ KB


# main

In [5]:
# объединенная ф-ция для извлечения полной информации из всего массива документов
def extractor(html_page) -> dict:
    html_dict = {'link_name': [], 'email': [], 'phone': [], 'link': [], 'other': []}
    
    # паттерны для регулярных выражений
    email_pattern = r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,8}'
    phone_pattern = r'\+?[789]{1,2}\s?\(?\d{3}\)?[-\s]?\d{3}[-\s]?\d{2}[-\s]?\d{2}'
    link_pattern = r'\bwww\.[\w-]+\.[A-Za-z]+'
    other_pattern = r'[А-Я]{2,6} \d{5,20}'
    
    # поиск совпадений по паттернам и отбор дубликатов
    emails = list(set(re.findall(email_pattern, html_page)))
    phones = list(set(re.findall(phone_pattern, html_page)))
    links = list(set(re.findall(link_pattern, html_page)))
    others = list(set(re.findall(other_pattern, html_page)))
    
    # проверка на пустое значение
    if not emails:
        emails = 0
    if not phones:
        phones = 0
    if not links:
        links = 0
    if not others:
        others = 0
        
    # добавление результатов в словарь    
    html_dict['email'] = emails
    html_dict['phone'] = phones
    html_dict['link'] = links
    html_dict['other'] = others
    
    return html_dict


In [6]:
# применим функцию для df и преобразуем словари на выходе в series. Результаты сохраним в новой таблице
contacts_df = df.iloc[:]['html_body'].apply(extractor).apply(pd.Series)

In [7]:
# добавим колонку с сайтами-референцами из df
contacts_df['html_site'] = df['html']

# отсортируем колонки
contacts_df = contacts_df[['html_site', 'email', 'phone', 'link', 'other']]

In [8]:
contacts_df.head(10)

,html_site,email,phone,link,other
0,http://0009.ru,[info@deviatichi.ru],"[8(499) 426-2506, 8(968) 602-7429]","[www.w3.org, www.deviatichi.ru]",0
1,http://001k.ru,0,"[94117647059, 784313725490, 96078431372, 94901...","[www.w3.org, www.famethemes.com]",0
2,http://003ms.ru,0,"[73853381051, +7 (937) 009-30-03, +79370093003...","[www.w3.org, www.liveinternet.ru, www.googleop...",0
3,http://003rt.ru,[Rating@Mail.ru],"[91309321397, 91508445431, 91808345700, 929081...","[www.googletagmanager.com, www.liveinternet.ru...",0
4,http://004.ru,0,"[89180131004, 90385206602, 89996399004, 978469...","[www.w3.org, www.facebook.com, www.instagram.com]",0
5,http://004sosh.siteedu.ru,[4.sosh@mail.ru],"[95217781444, +74953803067, +7 (928) 099-39-06...",[www.w3.org],0
6,http://005mebel.ru,"[info@studia70.ru, zakaz@005mebel.ru]","[+7 (812) 640-67-62, 74272248449, 74272250109,...","[www.bitrix24.ru, www.w3.org, www.google.com, ...",0
7,http://009.рф/about,"[direktor@cfk.ooo, support@009.am, manager4@cf...","[+79234770033, 81020000000, 70281052318, 81060...","[www.w3.org, www.googletagmanager.com, www.goo...","[ОГРН 1194205005653, ОКТМО 32731000001, КПП 42..."
8,http://009.рф/contacts,"[u003Esupport@009.am, support@009.am]","[+79234770033, 90283572338, +7 923 477 00 33]","[www.w3.org, www.googletagmanager.com, www.goo...",0
9,http://01-36.ru,0,"[88003007154, 88800000453, 875537821642, 84732...","[www.w3.org, www.google.com, www.google-analyt...",0


In [17]:
# запись в файл
contacts_df.to_csv('контактная_информация.csv',sep=';', index=False, encoding='UTF-16')

<h3> Algorithm </h3>

In [126]:
# функции-экстракторы контактной информации

def email_extractor(html_source) -> list:
    """экстрактор почты"""
    contact_info = []
    email_pattern = re.compile(r'[A-Za-z0-9._]+@.*?\.[A-Za-z]{2,8}')

    emails = re.findall(email_pattern, html_source)
    contact_info.extend(emails)
    
    if not contact_info:
        return 0
    return contact_info


def phone_extractor(html_source) -> list:
    """экстрактор номеров телефонов"""
    contact_info = []
    phone_pattern = re.compile(r'\+?[789]{1,2}\s?\(?\d{3}\)?[-\s]?\d{3}[-\s]?\d{2}[-\s]?\d{2}')
    
    phones = re.findall(phone_pattern, html_source)
    contact_info.extend(phones)
    
    if not contact_info:
        return 0
    return contact_info


def links_extractor(html_source) -> list:
    """экстрактор ссылок"""
    contact_info = []
    link_pattern = re.compile(r'\bwww\.\S*\.?\.[A-Za-z]+')
    
    link = re.findall(link_pattern, html_source)
    contact_info.extend(link)
    
    if not contact_info:
        return 0
    return contact_info


def other_extractor(html_source) -> list:
    """остальная релевантная информация"""
    contact_info = []
    other_pattern = re.compile(r'[А-Я]{2,6} \d{5,20}')
    
    other = re.findall(text=other_pattern)
    contact_info.extend(other)
    
    if not contact_info:
        return 0
    return contact_info
    

In [128]:
# тестовый запуск на мини-датасете
df_mini = df.iloc[:20]

# создание таблицы с контактной информацией
contacts_mini_df = pd.DataFrame(columns=['html_site','email', 'phone', 'links', 'other'])
contacts_mini_df['html_site'] = df_mini['html']

# применение функций 
contacts_mini_df['email'] = df_mini['html_body'].apply(email_extractor)
contacts_mini_df['phone'] = df_mini['html_body'].apply(phone_extractor)
contacts_mini_df['links'] = df_mini['html_body'].apply(links_extractor)
contacts_mini_df['other'] = df_mini['html_body'].apply(other_extractor)

contacts_mini_df

/var/folders/7n/rydsv3td2bb5j7n_24qcf5qc0000gn/T/ipykernel_64524/1452382906.py:44: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  other = soup.find_all(text=other_pattern)


,html_site,email,phone,links,other
1,http://001k.ru,0,"[+78442530707, 980392156862, 980392156862, 980...","[www.w3.org, www.w3.org, www.w3.org, www.w3.or...",0
2,http://003ms.ru,0,"[96378777213, 96378777213, +7 (499) 999-03-03,...","[www.w3.org, www.google-analytics.com/analytic...",0
3,http://003rt.ru,"[Rating@Mail.ru, Rating@Mail.ru]","[+78432220003, 92908183239, 92709023803, 92507...","[www.googletagmanager.com, www.google-analytic...",0
4,http://004.ru,0,"[82001632815, 86919293513, 88005000452, 891801...","[www.w3.org, www.w3.org, www.w3.org, www.w3.or...",0
5,http://004sosh.siteedu.ru,"[4.sosh@mail.ru, 4.sosh@mail.ru]","[+7 (928) 099-39-06, +74953803067, 95217781444...",[www.w3.org],0
6,http://005mebel.ru,"[info@studia70.ru, zakaz@005mebel.ru, zakaz@00...","[8 (812) 640-67-62, 74272247208, 799299551361,...","[www.w3.org, www.005mebel.ru, www.google.com/r...",0
7,http://009.рф/about,"[direktor@cfk.ooo, komdir@cfk.ooo, manager@cfk...","[94205005653, 70281080036, 81020000000, 702810...","[www.w3.org, www.googletagmanager.com, www.goo...","[ИНН 4253044486 КПП 425301001 , ОГРН 11942050..."
8,http://009.рф/contacts,"[support@009.am, support@009.am, support@009.a...","[90283572338, +79234770033, +7 923 477 00 33]","[www.w3.org, www.googletagmanager.com, www.goo...",0
9,http://01-36.ru,0,"[94603940302, 875537821642, 798322914332, 8800...","[www.w3.org, www.googletagmanager.com, www.gst...",0
10,http://01-klinika-21.ru,[21@yandex.ru],0,"[www.w3.org, www.ffoms.gov.ru]",0
